In [1]:
import numpy as np
import h5py
import sys
import pickle
from support import *
import astropy.cosmology as cosmo
import astropy.units as u
from astropy.cosmology import Planck15

import sys
#sys.path.append('/home/thomas.callister/CBC/effective-spin-priors/')
sys.path.append('../other_files/')
from priors import chi_effective_prior_from_isotropic_spins
from priors import joint_prior_from_isotropic_spins

In [2]:
import numpy as np
import scipy

def Joint_prob_Xeff_Xp(Xeff,Xp,q = 1,amax = 1,flag_debug = False):
    r'''
    Returns Joint prior on Effective Spin parameter Xeff and Precessing Spin Parameter Xp,
    given Mass ratio q, Max allowed spin magnitude amax, $p(Xeff,Xp|q,amax)$.
    Assumes $q>0$ and $0<amax<=1.$ $q>1$ inputs are interepreted as $1/q$.
    if flag_debug is True, then the function shows some integration for debugging and returns 4 integrals separately.
    '''
    assert (amax <= 1)*(amax > 0),"Invalid amax. 0<amax<=1 is accepted."
    
    def S(z):
        r'''
        Spence's function or dilogarithm Li_2(z).
        '''
        return scipy.special.spence(1-z)

    def g(x0,a0,b0):
        r'''
        Primitive function of log(x-bi)/(x-a-i) (x>=0).
        '''                    
        ans = np.zeros(len(a0))+0j
        case_special = (b0 == 0)*(x0 == 0)
        case_b_less_1 = (np.abs(b0) < 1)*(np.logical_not(case_special))
        case_b_1_a_less_0 = (b0 == 1)*(a0 <= 0)
        case_otherwise = np.logical_not(case_b_less_1|case_b_1_a_less_0|case_special)
        
        for j, case in enumerate([case_special,case_b_less_1,case_b_1_a_less_0,case_otherwise]):
            if np.any(case):
                x = x0[case]
                a = a0[case]
                b = b0[case]
                if j == 0:
                    ans[case] = 0
                if j == 1:
                    ans[case] = np.log(x-b*1j)*np.log((a-x+1j)/(a+1j-b*1j))+S((x-b*1j)/(a+1j-b*1j))
                if j == 2:
                    ans[case] = 1/2*(np.log(x-a-1j))**2+S(-a/(x-a-1j))
                if j == 3:
                    ans[case] = np.log(a+1j-b*1j)*np.log(a-x+1j)-S((a-x+1j)/(a+1j-b*1j))
        return ans
        
        

    def G(x0,a0,b0):
        r'''
        \int_0^x((x^2+b^2))/(1+(x-a)^2) dx.
        '''
        
        case_positive_x = (x0 > 0)
        case_negative_x = (x0 < 0)
        ans = np.zeros(len(x0))
        
        for j,case in enumerate([case_positive_x,case_negative_x]):
            if np.any(case):
                x = x0[case]
                a = a0[case]
                b = b0[case]
                if j == 0:
                    if flag_debug:
                        print("Given Values to H(x,a,b):")
                        print('x',x)
                        print('a',a)
                        print('b',b)
                        print('')
                    ans[case] = np.imag(g(x,a,b)+g(x,a,-b))-np.imag(g(np.zeros(len(x)),a,b)+g(np.zeros(len(x)),a,-b))
                if j == 1:
                    if flag_debug:
                        print('Avoid x<0 by transposing.')
                    ans[case] = -G(-x,-a,b)
        return ans

    def F(_x,_a,_b,_c,_d):
        r'''
        \int_0^x b*log((x^2+c^2)/d^2)/(b^2+(x-a)^2) dx.
        '''
        if np.any(_d == 0):
            print('Something is wrong!')
            raise ValueError
            
        x0 = np.atleast_1d(_x)
        a0 = np.atleast_1d(_a)
        b0 = np.atleast_1d(_b)
        c0 = np.atleast_1d(_c)
        d0 = np.atleast_1d(_d)
        max_len = len(b0)
        arr = np.zeros((5,max_len))
        for j, X in enumerate([x0,a0,b0,c0,d0]):
            if X.shape[0] == 1:
                arr[j,:] = X[0]
            else:
                arr[j,:] = X[:]
        x0,a0,b0,c0,d0 = arr
        
        if flag_debug:
            print(x0,a0,b0,c0,d0)
        ans = np.zeros(len(b0))
        case = (b0 != 0)
        x = x0[case]
        a = a0[case]
        b = b0[case]
        c = c0[case]
        d = d0[case]
        ans[case] = G(x/b,a/b,c/b) + np.log(b*b/d/d)*(np.arctan(a/b)+np.arctan((x-a)/b))
        return ans


    def Joint_prob_Xeff_Xp_1(_Xeff,_Xp,_q,flag_debug):
        r'''
        Returns Joint prior on Effective Spin parameter Xeff and Precessing Spin Parameter Xp, given Mass ratio q.
        Max allowed spin magnitude amax is assumed to be amax=1.
        '''
        Xp = np.atleast_1d(_Xp)
        Xeff = np.atleast_1d(_Xeff)
        q = np.atleast_1d(_q)
        assert np.min(q) > 0
        q[q>1] = 1/q[q>1]
    
        zeta = (1+q)*Xeff
        nu = (4+3*q)/(3+4*q)
    
        max_len = np.max([Xp.shape[0],Xeff.shape[0],q.shape[0]])
        arr = np.zeros((5,max_len))
        for j, X in enumerate([Xeff,Xp,q,zeta,nu]):
            if X.shape[0] == 1:
                arr[j,:] = X[0]
            else:
                arr[j,:] = X[:]
            
        Xeff0,Xp0,q0,zeta0,nu0 = arr
        threshold = q0/nu0
    
        case_OoB = np.logical_or((Xp0 <= 0),(Xp0 >= 1))
        case_lsr_than_threshold = np.logical_and(np.logical_not(case_OoB),(Xp0 < threshold))
        case_gtr_than_threshold = np.logical_and(np.logical_not(case_OoB),(Xp0 >= threshold))
    
        #Joint PDF is a sum of 4 integrals
        int_1 = np.zeros(max_len)
        int_2 = np.zeros(max_len)
        int_3 = np.zeros(max_len)
        int_4 = np.zeros(max_len)
    
        for j,case in enumerate([case_OoB,
                                case_lsr_than_threshold,
                                case_gtr_than_threshold]):
            if np.any(case):
                Xeff = Xeff0[case]
                Xp = Xp0[case]
                q = q0[case]
                zeta = zeta0[case]
                nu = nu0[case]
            
                if j != 0:
                #Only int_2 have non-zero value for j = 2
                    max_2 = np.minimum(zeta+np.sqrt(1-Xp*Xp),q)
                    min_2 = np.maximum(zeta-np.sqrt(1-Xp*Xp),-q)
                    zero = (min_2 > max_2)
                    if np.any(zero):
                        max_2[zero] = min_2[zero]
                    if flag_debug:
                        print('Calculating 2nd integral for j = {}'.format(j))
                        print('x_max:',max_2)
                        print('x_min:',min_2)
                    int_2[case] = -(1+q)/8/q * (F(max_2,zeta,Xp,0,q)-F(min_2,zeta,Xp,0,q))
                
                #int_1,int_3,int_4 have none-zero value only if j = 1
                    if j == 1:
                    
                        #int_1
                        max_1 = np.minimum(zeta+np.sqrt(1-Xp*Xp),np.sqrt(q*q-nu*nu*Xp*Xp))
                        min_1 = np.maximum(zeta-np.sqrt(1-Xp*Xp),-np.sqrt(q*q-nu*nu*Xp*Xp))
                        zero = (min_1 > max_1)                            
                        if np.any(zero):
                            max_1[zero] = min_1[zero]
                        if flag_debug:
                            print('Calculating 1st integral')
                            print('x_max:',max_1)
                            print('x_min:',min_1)
                        int_1[case] = (1+q)/8/q * (F(max_1,zeta,Xp,nu*Xp,q)-F(min_1,zeta,Xp,nu*Xp,q))
                    
                        #int_3
                        max_3 = np.minimum(zeta+np.sqrt(q*q-nu*nu*Xp*Xp),np.sqrt(1-Xp*Xp))
                        min_3 = np.maximum(zeta-np.sqrt(q*q-nu*nu*Xp*Xp),-np.sqrt(1-Xp*Xp))
                        zero = (min_3 > max_3)
                        if np.any(zero):
                            max_3[zero] = min_3[zero]
                        if flag_debug:
                            print('Calculating 3rd integral')
                            print('x_max:',max_3)
                            print('x_min:',min_3)
                        int_3[case] = (1+q)*nu/8/q * (F(max_3,zeta,nu*Xp,Xp,1)-F(min_3,zeta,nu*Xp,Xp,1))
                    
                        #int_4
                        max_4 = np.minimum(zeta+np.sqrt(q*q-nu*nu*Xp*Xp),q/q)
                        min_4 = np.maximum(zeta-np.sqrt(q*q-nu*nu*Xp*Xp),-q/q)
                        zero = (min_4 > max_4)
                        if np.any(zero):
                            max_4[zero] = min_4[zero]
                        if flag_debug:
                            print('Calculating 4th integral')
                            print('x_max:',max_4)
                            print('x_min:',min_4)
                        int_4[case] = -(1+q)*nu/8/q * (F(max_4,zeta,nu*Xp,0,1)-F(min_4,zeta,nu*Xp,0,1))
                    
        if flag_debug:
            return int_1,int_2,int_3,int_4
        elif max_len == 1:
            return (int_1+int_2+int_3+int_4)[0]
        return int_1+int_2+int_3+int_4
    
    if flag_debug:
        int_1,int_2,int_3,int_4 = Joint_prob_Xeff_Xp_1(Xeff/amax,Xp/amax,q,flag_debug)
        return int_1/amax/amax,int_2/amax/amax,int_3/amax/amax,int_4/amax/amax
    return Joint_prob_Xeff_Xp_1(Xeff/amax,Xp/amax,q,flag_debug)/amax/amax


In [6]:
def loadInjectionsAlt(ifar_threshold,snr_threshold):

    #mockDetections = h5py.File('/home/reed.essick/rates+pop/o1+o2+o3-sensitivity-estimates/LIGO-T2100377-v2/o1+o2+o3_bbhpop_real+semianalytic-LIGO-T2100377-v2.hdf5','r')
    mockDetections = h5py.File('../other_files/o1+o2+o3_bbhpop_real+semianalytic-LIGO-T2100377-v2.hdf5','r')

    ifar_1 = mockDetections['injections']['ifar_gstlal'][()]
    ifar_2 = mockDetections['injections']['ifar_pycbc_bbh'][()]
    ifar_3 = mockDetections['injections']['ifar_pycbc_hyperbank'][()]
    snr = mockDetections['injections']['optimal_snr_net'][()]
    nTrials = mockDetections.attrs['total_generated']

    detected_O3 = np.where((ifar_1>ifar_threshold) | (ifar_2>ifar_threshold) | (ifar_3>ifar_threshold))[0]
    detected_O1O2 = np.where((mockDetections['injections']['name'][()]!=b'o3') & (snr>snr_threshold))[0]
    detected_full = np.concatenate([detected_O1O2,detected_O3])
    print(detected_O3.size,detected_O1O2.size,detected_full.size)

    m1_det = np.array(mockDetections['injections']['mass1_source'][()])[detected_full]
    m2_det = np.array(mockDetections['injections']['mass2_source'][()])[detected_full]
    s1x_det = np.array(mockDetections['injections']['spin1x'][()])[detected_full]
    s1y_det = np.array(mockDetections['injections']['spin1y'][()])[detected_full]
    s1z_det = np.array(mockDetections['injections']['spin1z'][()])[detected_full]
    s2x_det = np.array(mockDetections['injections']['spin2x'][()])[detected_full]
    s2y_det = np.array(mockDetections['injections']['spin2y'][()])[detected_full]
    s2z_det = np.array(mockDetections['injections']['spin2z'][()])[detected_full]
    z_det = np.array(mockDetections['injections']['redshift'][()])[detected_full]

    precomputed_p_m1m2z_spin = np.array(mockDetections['injections']['sampling_pdf'][()])[detected_full]
    p_inj_spin1 = (1./(4.*np.pi))*(1./0.998)/(s1x_det**2+s1y_det**2+s1z_det**2)
    p_inj_spin2 = (1./(4.*np.pi))*(1./0.998)/(s2x_det**2+s2y_det**2+s2z_det**2)
    precomputed_p_m1m2z = precomputed_p_m1m2z_spin/p_inj_spin1/p_inj_spin2

    runs = np.ones(m1_det.size)

    return m1_det,m2_det,s1x_det,s1y_det,s1z_det,s2x_det,s2y_det,s2z_det,z_det,precomputed_p_m1m2z,runs,nTrials

def genInjectionFile(ifar_threshold,snr_threshold,filename,mode='K'):

    # Load
    m1_det,m2_det,s1x_det,s1y_det,s1z_det,s2x_det,s2y_det,s2z_det,z_det,p_draw_m1m2z,runs,nTrials = loadInjectionsAlt(ifar_threshold,snr_threshold)

    # Derived parameters
    q_det = m2_det/m1_det
    Xeff_det = (m1_det*s1z_det + m2_det*s2z_det)/(m1_det+m2_det)
    Xp_det = calculate_Xp(s1x_det,s1y_det,s1z_det,s2x_det,s2y_det,s2z_det,q_det)
    a1_det = np.sqrt(s1x_det**2 + s1y_det**2 + s1z_det**2)
    a2_det = np.sqrt(s2x_det**2 + s2y_det**2 + s2z_det**2)
    cost1_det = s1z_det/a1_det
    cost2_det = s2z_det/a2_det

    # Compute marginal draw probabilities for chi_effective and joint chi_effective vs. chi_p probabilities
    p_draw_xeff = np.zeros(Xeff_det.size)
    p_draw_xeff_xp = np.zeros(Xeff_det.size)
    if mode=='K':
        for i in range(p_draw_xeff.size):
            if i%500==0:
                print('{}/{} finished'.format(i,p_draw_xeff.size))
            p_draw_xeff[i] = chi_effective_prior_from_isotropic_spins(q_det[i],1.,Xeff_det[i])[0]
            p_draw_xeff_xp[i] = joint_prior_from_isotropic_spins(q_det[i],1.,Xeff_det[i],Xp_det[i],ndraws=10000)[0]
    else:
        for i in range(p_draw_xeff.size):
            if i%500==0 or (i+1) == p_draw_xeff.size:
                print('{}/{} finished'.format(i+1,p_draw_xeff.size))
            p_draw_xeff[i] = chi_effective_prior_from_isotropic_spins(q_det[i],1.,Xeff_det[i])[0]
        p_draw_xeff_xp = Joint_prob_Xeff_Xp(Xeff_det,Xp_det,q_det,amax = 1,flag_debug = False)

    # Combine
    pop_reweight = 1./(p_draw_m1m2z*p_draw_xeff_xp)
    pop_reweight_XeffOnly = 1./(p_draw_m1m2z*p_draw_xeff)
    pop_reweight_noSpin = 1./p_draw_m1m2z

    # Also compute factors of dVdz that we will need to reweight these samples during inference later on
    dVdz = 4.*np.pi*Planck15.differential_comoving_volume(z_det).to(u.Gpc**3*u.sr**(-1)).value

    # Store and save
    injectionDict = {
            'm1':m1_det,
            'm2':m2_det,
            'Xeff':Xeff_det,
            'Xp':Xp_det,
            'z':z_det,
            's1z':s1z_det,
            's2z':s2z_det,
            'a1':a1_det,
            'a2':a2_det,
            'cost1':cost1_det,
            'cost2':cost2_det,
            'dVdz':dVdz,
            'weights':pop_reweight,
            'weights_XeffOnly':pop_reweight_XeffOnly,
            'weights_noSpin':pop_reweight_noSpin,
            'nTrials':nTrials,
            'obsRuns':runs
            }

    with open(filename,"wb") as f:
        pickle.dump(injectionDict,f,protocol=2)

if __name__=="__main__":

    genInjectionFile(1.,10.,'./injectionDict_rerun_directMixture_FAR_1_in_1_K.pickle',mode='K')
    genInjectionFile(1.,10.,'./injectionDict_rerun_directMixture_FAR_1_in_1_A.pickle',mode='A')
    #genInjectionFile(5.,10.,'./injectionDict_10-20_directMixture_FAR_1_in_5.pickle')


34256 5504 39760
0/39760 finished
500/39760 finished
1000/39760 finished
1500/39760 finished
2000/39760 finished
2500/39760 finished
3000/39760 finished
3500/39760 finished
4000/39760 finished
4500/39760 finished
5000/39760 finished
5500/39760 finished
6000/39760 finished
6500/39760 finished
7000/39760 finished
7500/39760 finished
8000/39760 finished
8500/39760 finished
9000/39760 finished
9500/39760 finished
10000/39760 finished
10500/39760 finished
11000/39760 finished
11500/39760 finished
12000/39760 finished
12500/39760 finished
13000/39760 finished
13500/39760 finished
14000/39760 finished
14500/39760 finished
15000/39760 finished
15500/39760 finished
16000/39760 finished
16500/39760 finished
17000/39760 finished
17500/39760 finished
18000/39760 finished
18500/39760 finished
19000/39760 finished
19500/39760 finished
20000/39760 finished
20500/39760 finished
21000/39760 finished
21500/39760 finished
22000/39760 finished
22500/39760 finished
23000/39760 finished
23500/39760 finished

In [4]:
injection_K = np.load('./injectionDict_rerun_directMixture_FAR_1_in_1.pickle',allow_pickle=True)
injection_K

{'m1': array([30.94872788, 33.07151707, 93.19263176, ..., 13.72910118,
        29.98383522, 36.74379349]),
 'm2': array([28.74143435, 21.72843901, 79.26658633, ...,  5.84848452,
        23.55150223, 21.63918686]),
 'Xeff': array([-0.02523325, -0.03246811,  0.06974145, ..., -0.35393217,
         0.13539099, -0.22754405]),
 'Xp': array([0.88060233, 0.13491056, 0.114209  , ..., 0.52087009, 0.10170022,
        0.19887426]),
 'z': array([0.15632872, 0.20204913, 0.85555993, ..., 0.16707543, 0.39559993,
        0.19040988]),
 's1z': array([-0.04701484, -0.02031199,  0.10934089, ..., -0.38111317,
        -0.0034426 , -0.27684465]),
 's2z': array([-0.00177887, -0.0509702 ,  0.02318494, ..., -0.29012579,
         0.31214249, -0.1438306 ]),
 'a1': array([0.4063689 , 0.02328626, 0.1456331 , ..., 0.6454091 , 0.00579056,
        0.34087231]),
 'a2': array([0.95830447, 0.22373608, 0.13935026, ..., 0.74103707, 0.33968898,
        0.36900714]),
 'cost1': array([-0.11569498, -0.87227346,  0.75079701, ..